- Requirements.txt should contain the following (other modules are included in the standard Python library):
    - requests
    - pandas
    - pytz

In [14]:
################################################################################
# make these imports into a set of requirements (see above)
################################################################################

import requests
import pandas as pd
from datetime import date, timedelta, datetime
import pytz
import json

In [15]:
################################################################################
# set and create some variables
################################################################################

# Coordinates for [location]: 
latitude = '55.951009'
longitude = '-3.100191'

# Set the start date to current date:
start_date = date.today()

In [16]:
################################################################################
# function to fetch sunrise and sunset data
################################################################################

def fetch_sunrise_sunset_data(date_str):
    """Fetch sunrise and sunset times from API."""
    url = f'https://api.sunrise-sunset.org/json?lat={latitude}&lng={longitude}&date={date_str}&formatted=0'
    response = requests.get(url)

    # Return JSON data if request is successful, else print error and return None
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching sunrise/sunset data: {response.status_code}")
        return None

################################################################################
# function to convert times to local
################################################################################

def convert_to_bst(time_str):
    """Convert GMT time string to BST."""
    gmt = pytz.timezone('GMT')
    bst = pytz.timezone('Europe/London')
    gmt_time = datetime.fromisoformat(time_str).replace(tzinfo=gmt)
    bst_time = gmt_time.astimezone(bst)
    return bst_time.strftime('%H:%M')

################################################################################
# use the functions to fetch times for one week
################################################################################

sun_data = []

for i in range(7):
    # Calculate the date for each day in the range
    current_date = start_date + timedelta(days=i)
    date_str = current_date.isoformat()

    # Fetch sunrise and sunset data for the current date
    sun_info = fetch_sunrise_sunset_data(date_str)

    # If data is successfully fetched, convert times to BST and add to sun_data
    if sun_info and 'results' in sun_info:
        sun_data.append({
            'date': date_str,
            'sunrise': convert_to_bst(sun_info['results']['sunrise']),
            'sunset': convert_to_bst(sun_info['results']['sunset'])
        })
    else:
        print(f"No sun data fetched for date: {date_str}")


In [17]:
################################################################################
# put the data in a dataframe
################################################################################

df = pd.DataFrame(sun_data)


In [18]:
################################################################################
# use the date column as the index
################################################################################

df.set_index('date', inplace=True)

In [19]:
# convert the index to datetime
df.index = pd.to_datetime(df.index)

# create a new column for the day of the week
df['day_of_week'] = df.index.day_name()

# Define the new order of the columns with 'day_of_week' at the beginning
column_order = ['day_of_week', 'sunrise', 'sunset']

# Reorder the DataFrame
df = df[column_order]

In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = 'https://www.tidetime.org/europe/united-kingdom/portobello.htm'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table in the soup.
table = soup.find('table', {'id': 'tideTable'})

# Find all the table rows in the table.
rows = table.find_all('tr')

# Initialize an empty list to hold the data.
data = []

# Initialize a counter for the day of the week.
day = 0

# Loop through each row.
for row in rows:
    # Find all the table data cells in the row.
    cells = row.find_all('td')

    # Loop through each cell.
    for cell in cells:
        # Increment the day counter.
        day += 1

        # Find all the high and low tides in the cell.
        tides = cell.find_all('li', {'class': ['highTide', 'lowTide']})

        # Loop through each tide.
        for tide in tides:
            # Get the tide type, time and height from the tide text.
            tide_type = tide.find('span', {'class': 'tidal-state'}).text
            tide_time = tide.find('strong').text.replace(tide_type, '').strip()
            tide_height = tide.text.split(')')[0].split('(')[-1]

            # Add the day, tide type, time and height to the data list.
            data.append([day, tide_type, tide_time, tide_height])

# Convert the data list into a DataFrame.
df_tides = pd.DataFrame(data, columns=['Day', 'Tide Type', 'Tide Time', 'Tide Height'])

print(df_tides)


    Day Tide Type Tide Time Tide Height
0     1      High   06:07am       5.28m
1     1       Low   12:24pm       0.88m
2     1      High   06:56pm       5.18m
3     2       Low   12:47am       1.51m
4     2      High   07:01am       5.16m
5     2       Low   01:25pm          1m
6     2      High   07:53pm       5.02m
7     3       Low   01:48am        1.7m
8     3      High   07:59am       5.03m
9     3       Low   02:30pm       1.11m
10    3      High   08:58pm       4.88m
11    4       Low   02:50am       1.82m
12    4      High   09:05am       4.92m
13    4       Low   03:31pm       1.19m
14    4      High   10:02pm       4.79m
15    5       Low   03:49am       1.85m
16    5      High   10:14am       4.87m
17    5       Low   04:39pm       1.21m
18    5      High   11:07pm       4.77m
19    6       Low   04:53am       1.79m
20    6      High   11:21am        4.9m
21    6       Low   05:46pm        1.2m
22    7      High   12:08am       4.83m
23    7       Low   05:57am       1.65m


- DONE - Add a Day Of The Week column

- DONE - Fetch the Tides webpage using Python's requests library.

- DONE - Parse the HTML: use Beautiful Soup to parse the HTML and find the table containing the tide data.

- DONE - Extract and structure the data: go through cell by cell to extract the tide data and put it into a structured format. This might involve creating a Pandas DataFrame, or perhaps a more specialized data structure that's suited to your needs.

- Convert to the desired format: If necessary, you may need to convert the times to a different format, or perform other transformations on the data to prepare it for your analysis.

- py_Tides_old_remains contains the old bits of code for weather and visualisation.  They won't map directly but will hopefully offer some guidance on what remains to do.